In [1]:
import torch, sys
sys.path.insert(0, '../')
from my_utils import gpu_utils
import importlib
import gc
from my_utils import align_utils as autils, utils
from my_utils.alignment_features import *
from tqdm import tqdm
run = 0
from tqdm import tqdm
from gnn_utils import eval_utils

In [2]:
# !pip install torch-geometric
# !pip install tensorboardX

# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

#  print(torch.version.cuda)
#  print(torch.__version__)    

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
free_gpu1 = '6'
free_gpu2 = '5'


In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt




In [4]:
import argparse
from multiprocessing import Pool

# set random seed
config_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc-ui-demo/config_pbc.ini"
utils.setup(config_file)

params = argparse.Namespace()
#params.editions_file =  "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/eng_fra_pbc/lang_list.txt"

#params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/eng_fra_pbc/eng-fra.gold"
#params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-heb-fin-gold-alignments_test.txt"

params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-grc-fin-gold-alignments_train.txt"
pros, surs = autils.load_gold(params.gold_file)
all_verses = list(pros.keys())
params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-heb-fin-gold-alignments_train.txt"
pros, surs = autils.load_gold(params.gold_file)
all_verses.extend(list(pros.keys()))
all_verses = list(set(all_verses))
print(len(all_verses))

params.editions_file =  "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi_lang_list.txt"
editions, langs = autils.load_simalign_editions(params.editions_file)
current_editions = [editions[lang] for lang in langs]

def get_pruned_verse_alignments(args):
    verse, current_editions = args
    print(verse)
    verse_aligns_inter = autils.get_verse_alignments(verse)
    verse_aligns_gdfa = autils.get_verse_alignments(verse, gdfa=True)

    autils.prune_non_necessary_alignments(verse_aligns_inter, current_editions)
    autils.prune_non_necessary_alignments(verse_aligns_gdfa, current_editions)

    gc.collect()
    return verse_aligns_inter, verse_aligns_gdfa
    

verse_alignments_inter = {}
verse_alignments_gdfa = {}
args = []
for i,verse in enumerate(all_verses):
    args.append((verse, current_editions[:]))

print('going to get alignments')
with Pool(100) as p:
    all_res = p.map(get_pruned_verse_alignments, args)

for i,verse in enumerate(all_verses):
    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
    verse_alignments_inter[verse] = verse_aligns_inter
    verse_alignments_gdfa[verse] = verse_aligns_gdfa

for verse in all_verses[:]:
    if len(verse_alignments_inter[verse].keys()) < 10:
        all_verses.remove(verse)
        
utils.LOG.info("done")

24159
going to get alignments
5800902116009001050340072401700412025023
310000040901500843005041
24026007

23028013




2200501012019037



4200301701043010
270120111908002005021005
24026001110140172101200702023016

1700901643020019



24024007

2002000740018001
660040101300203019134003400080201904401701035003
19029003

03013029
26035002
03004034

02031013
24006009




1300700925001017430080270501901066003002




04021035
6600700704018025


400190114000902803007028


66014019
14030002
23037013
1001202058002010

44013045
04036009
13021016
26033019
11018002
14004005
19006002
10001006
36001008
01028021
23014017
23014027
18024010
42022056
19075011
05010003
09010012
42014004
58007001
07005010
05014015
46007037
04026009
40001012
24026005
02023019
04016014
14017017
01004006
18005004
06022015
19027006
25001020
19063010
23003007
42017033
09002011
60003001
48003028
19132008
05005033
44013015
28011009
09016014
09028024
14036021
55004001
47012014
06007015
42009041
13008015
04019005
19073028
1302200

Process ForkPoolWorker-28:
Traceback (most recent call last):
  File "/mounts/Users/student/ayyoob/anaconda3/envs/multalign_graph/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/mounts/Users/student/ayyoob/anaconda3/envs/multalign_graph/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/mounts/Users/student/ayyoob/anaconda3/envs/multalign_graph/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/mounts/Users/student/ayyoob/anaconda3/envs/multalign_graph/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-4-2f960d012a44>", line 30, in get_pruned_verse_alignments
    verse_aligns_inter = autils.get_verse_alignments(verse)
  File "../my_utils/align_utils.py", line 69, in get_verse_alignments
    for line in f:
  File "/mounts/Users/student/ayyoob/anaconda3/envs/multalig

KeyboardInterrupt: 

In [ ]:
class DeeperGCN(torch.nn.Module):
    def __init__(self,node_dim, edge_dim, hidden_channels, num_layers, features):
        super(DeeperGCN, self).__init__()

        self.node_encoder = nn.Linear(node_dim, hidden_channels)
        self.edge_encoder = nn.Linear(edge_dim, hidden_channels)

        self.layers = torch.nn.ModuleList()
        for i in range(1, num_layers + 1):
            conv = pyg_nn.GENConv(hidden_channels, hidden_channels, aggr='softmax',
                           t=1.0, learn_t=True, num_layers=2, norm='layer')
            norm = nn.LayerNorm(hidden_channels, elementwise_affine=True)
            act = nn.ReLU(inplace=True)

            layer = pyg_nn.DeepGCNLayer(conv, norm, act, block='res+', dropout=0.1,
                                 ckpt_grad=i % 3)
            self.layers.append(layer)

        self.already_inited = False
        self.prev_edge_index = None
        self.prev_edge_attr = None

        self.feature_encoder = FeatureEncoding(features)
        
    def forward(self, x, edge_index):
        x = self.feature_encoder(x)
        x = self.node_encoder(x)

        if not self.already_inited or self.prev_edge_index.data_ptr() != edge_index.data_ptr():
            edge_index_np = edge_index.cpu().numpy()
            val_indices = x_edge_np[edge_index_np[0, :], edge_index_np[1, :]]
            vals = x_edge_vals[val_indices, :]
            vals = vals.reshape((vals.shape[1], vals.shape[2]))
            self.prev_edge_attr = vals.to(dev)
            self.prev_edge_index = edge_index
            self.already_inited = True
        edge_attr = self.edge_encoder(self.prev_edge_attr)

        x = self.layers[0].conv(x, edge_index, edge_attr)

        for layer in self.layers[1:]:
            x = layer(x, edge_index, edge_attr)

        x = self.layers[0].act(self.layers[0].norm(x))
        # x = F.dropout(x, p=0.1, training=self.training)

        return x

In [ ]:
  
import numpy as np

class TruncatedLoss(nn.Module):

    def __init__(self, q=0.7, k=0.5, trainset_size=50000):
        super(TruncatedLoss, self).__init__()
        self.q = q
        self.k = k
        self.weight = torch.nn.Parameter(data=torch.ones(trainset_size, 1), requires_grad=False)
             
    def forward(self, logits, targets, indexes):
        indexes = torch.from_numpy(indexes)
        p = F.softmax(logits, dim=1)
        Yg = torch.gather(p, 1, torch.unsqueeze(targets, 1))

        loss = ((1-(Yg**self.q))/self.q)*self.weight[indexes]
        loss = torch.mean(loss)

        return loss

    def update_weight(self, logits, targets, indexes):
        indexes = torch.from_numpy(indexes).to(torch.int64)
        p = F.softmax(logits, dim=1)
        Yg = torch.gather(p, 1, torch.unsqueeze(targets, 1))
        Lq = ((1-(Yg**self.q))/self.q)
        
        Lqk = np.repeat(((1-(self.k**self.q))/self.q), targets.size(0))
        Lqk = torch.from_numpy(Lqk).type(torch.cuda.FloatTensor)
        Lqk = torch.unsqueeze(Lqk, 1)
        condition = torch.gt(Lqk, Lq)
        self.weight[indexes] = condition.type(torch.cuda.FloatTensor)
    
    def update_weight2(self, logits, targets, indexes, min = 0.4, max=0.6):
        indexes = torch.from_numpy(indexes)
        p = F.softmax(logits, dim=1)
        a = torch.gather(p, 1, torch.unsqueeze(targets, 1))
        c = a < min 
        d = a > max
        self.weight[indexes] = c.type(torch.cuda.FloatTensor) + d.type(torch.cuda.FloatTensor)



In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, features, n_head = 2, edge_feature_dim = 0,):
        super(Encoder, self).__init__()
        self.lin = nn.Linear(in_channels, out_channels)
        self.conv1 = pyg_nn.GATConv(in_channels, 2* out_channels, heads= n_head) 
        self.conv2 = pyg_nn.GATConv(2 * n_head *  out_channels , int(out_channels), heads= n_head)
        #self.conv3 = pyg_nn.GATConv(2 * n_head *  out_channels , int(out_channels), heads= n_head)
        self.conv_gin = pyg_nn.GINConv(
            nn.Sequential(nn.Linear(2*out_channels, 2*out_channels), nn.BatchNorm1d(2*out_channels), nn.ReLU(),
                       nn.Linear(2*out_channels, 2*out_channels), nn.ReLU()))
        # self.conv1 = pyg_nn.SuperGATConv(in_channels, 2 * out_channels, heads= n_head) 
        # self.conv2 = pyg_nn.SuperGATConv(n_head * 2 * out_channels, out_channels, heads= n_head)
        
        self.feature_encoder = FeatureEncoding(features)

        self.already_inited = False
        self.prev_edge_index = None
        self.prev_edge_attr = None

    def forward(self, x, edge_index):
        x = self.feature_encoder(x, dev)
        #if not self.already_inited or self.prev_edge_index.data_ptr() != edge_index.data_ptr():
        #    edge_index_np = edge_index.cpu().numpy()
        #    val_indices = x_edge_np[edge_index_np[0, :], edge_index_np[1, :]]
        #    vals = x_edge_vals[val_indices, :]
        #    vals = vals.reshape((vals.shape[1], vals.shape[2]))
        #    self.prev_edge_attr = vals.to(dev)
        #    self.prev_edge_index = edge_index
        #    self.already_inited = True
        #x = self.lin(x)
        x = F.elu(self.conv1(x, edge_index, ))
        x = self.conv_gin(x, edge_index)
        return self.conv2(x, edge_index)#, self.conv3(x, edge_index)
    
def randomize_add_noise(train_pos_edge_index, train_neg_edge_index):
    pos_len = train_pos_edge_index.shape[1]
    neg_len = train_neg_edge_index.shape[1]

    pos_res_len = int((pos_len/2) - (pos_len/2) * pos_noise)
    neg_res_len = int((neg_len/2) - (neg_len/2) * neg_noise)

    all_pos = torch.arange(pos_len).reshape((int(pos_len/2), 2))
    pos_keep = torch.randperm(int(pos_len/2))[:pos_res_len]
    pos_keep = torch.sort(pos_keep).values
    pos_pos_index = torch.flatten(all_pos[pos_keep, :])

    all_neg = torch.arange(neg_len).reshape((int(neg_len/2), 2))
    neg_keep = torch.randperm(int(neg_len/2))[neg_res_len:]
    neg_keep = torch.sort(neg_keep).values
    neg_pos_index = torch.flatten(all_neg[neg_keep, :])

    pos_res = torch.cat((train_pos_edge_index[:, pos_pos_index], train_neg_edge_index[:, neg_pos_index]), dim=1)
    print('orig_shape', train_pos_edge_index.shape)
    print('final shape', pos_res.shape)

    return pos_res    

def train(epoch):
    global optimizer, model, decoder, criterion, batch_size, targets, model2
    total_loss = 0
    tot_l1, tot_reg = 0,0
    model.train()

    train_pos_edge_index_permed = train_pos_edge_index[:, torch.randperm(train_pos_edge_index.shape[1])]
    train_neg_edge_index_permed = train_neg_edge_index[:, torch.randperm(train_neg_edge_index.shape[1])]
    # edge_index_seq_sent_neg_permed = edge_index_seq_sent_neg[:, torch.randperm(edge_index_seq_sent_neg.shape[1])]
    # edge_index_seq_sent_permed = edge_index_seq_sent[:, torch.randperm(edge_index_seq_sent.shape[1])]

    neg_pos_coeff = train_neg_edge_index.shape[1]/ train_pos_edge_index_permed.shape[1]
    # seq_coeff = edge_index_seq_sent.shape[1]/ train_pos_edge_index_permed.shape[1]
    # seq_coeff_neg = edge_index_seq_sent_neg.shape[1]/ train_pos_edge_index_permed.shape[1]

    if (epoch) >= start_prune and (epoch) % 7 == 0:
        print('updating weights')
        torch.cuda.set_device(div2)  

        #gc.collect()
        #with torch.no_grad():
        #    torch.cuda.empty_cache()

        model2 = torch.load('/mounts/work/ayyoob/models/gnn/checkpoint/ckpt.t7.' + str(run), map_location=lambda storage, loc: storage.cuda(div2))['net']
        
        targets = targets.to(dev)
        model2.eval()
        z = model2.encode(x.to(dev), train_pos_edge_index.to(dev)) 
        criterion = criterion.to(dev)
        batch_size *= 5
        optimizer.zero_grad()
        for i in tqdm(range(int(train_pos_edge_index.shape[1]/batch_size)+1)):
            input = train_pos_edge_index[:,int(i*batch_size):int(batch_size*(i+1))].to(dev)
            output = model2.decode(z, input)
            indexes = model2.decoder.val_indices
            tgt = torch.ones(input.shape[1], dtype=torch.int64).to(dev)
            criterion.update_weight(output, tgt, indexes)
        #for i in tqdm(range(int(indices_np.shape[1]/batch_size)+1)):
        #    input = indices_np[:, int(i*batch_size):int((i+1)*batch_size)]
        #    output = model2.decode(z, torch.from_numpy(input).to(dev))
        #    indexes = np.arange(int(i*batch_size),int((i+1)*batch_size))
        #    if input.shape[1] < indexes.shape[0]:
        #        indexes = indexes[:input.shape[1]]
            
        #    tgt = targets[indexes]
        #    criterion.update_weight2(output, tgt, indexes)

        model2 = None
        
        torch.cuda.set_device(div1) 
        criterion = criterion.to(dev)
        batch_size /= 5
        targets = targets.to(dev)

        gc.collect()
        with torch.no_grad():
            torch.cuda.empty_cache()
    

    
    for i in tqdm(range(int(train_pos_edge_index_permed.shape[1]/batch_size)+1)):
        optimizer.zero_grad()
        z = model.encode(x, train_pos_edge_index) 
        loss1 = recon_loss(model, z, train_pos_edge_index_permed[:, int(i*batch_size):int((i+1)*batch_size)],
                     train_neg_edge_index_permed[:, int(i*batch_size):int((i+1)*batch_size)]) #TODO try providing better neg edges
       
        loss = loss1 #+ (1 / x.shape[0]) * model.kl_loss() #+ reg_loss #
        tot_l1 += loss1
        #tot_reg += reg_loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        #if epoch > 0:
        #    if i  % 80 == 0:
        #        alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses, test_nodes_map,
        #                dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer)
        #        # decoder.set_objective('sequence_prediction')
        #        # auc, ap = test(edge_index_seq_sent, edge_index_seq_sent_neg, epoch)
        #        # print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
        #        # if epoch > 4:
        #        #     decoder.set_objective('link_prediction')
        #        model.train()
        
        if (i+1)*batch_size > train_pos_edge_index_permed.shape[1]:
            break
    print(f'classfication loss, regression loss: ', tot_l1.item())#, tot_reg.item())
    #print(f"sigmas: ", model.decoder.log_vars)
    writer.add_scalar("loss", total_loss, epoch)
    # print(f"train loss: {loss.item()}")

def test(pos_edge_index, neg_edge_index, epoch):
    model.eval()
    tot_auc = tot_ap = 0
    with torch.no_grad():
        z = model.encode(x_test, torch.cat((train_pos_edge_index, neg_edge_index), dim=1))
        
        neg_pos_coeff = neg_edge_index.shape[1]/ pos_edge_index.shape[1]
        for i in (range(int(pos_edge_index.shape[1]/batch_size)+1)):
            auc,ap = model.test(z, pos_edge_index[:, i*batch_size:(i+1)*batch_size],
                                neg_edge_index[:, int(i*batch_size*neg_pos_coeff):int((i+1)*batch_size*neg_pos_coeff)])

            tot_auc += auc * pos_edge_index[:, i*batch_size:(i+1)*batch_size].shape[1]
            tot_ap += ap *  pos_edge_index[:, i*batch_size:(i+1)*batch_size].shape[1]


    return tot_auc/pos_edge_index.shape[1], tot_ap/pos_edge_index.shape[1]



In [ ]:
from torch_geometric.utils import (negative_sampling, remove_self_loops,
                                   add_self_loops)
EPS = 1e-15

def recon_loss(model, z, pos_edge_index, neg_edge_index=None):
    target_len = (pos_edge_index.shape[1], neg_edge_index.shape[1])
    if target_len not in target_tensors:
        res = torch.zeros(target_len[0]+target_len[1], dtype=torch.int64).to(dev)
        res[:target_len[0]] = 1
        #res = torch.zeros((target_len[0]+target_len[1], 2), dtype=torch.float).to(dev)
        #res[:target_len[0], 1] = 1
        #res[target_len[0]:, 0] = 1

        target_tensors[target_len] = res

    logits = model.decoder(z, torch.cat((pos_edge_index, neg_edge_index), dim=1), sigmoid=False) # I think for truncated loss we should not do the sigmoid
    indexes = model.decoder.val_indices
    # Do not include self-loops in negative samples
    #pos_edge_index, _ = remove_self_loops(pos_edge_index)
    #pos_edge_index, _ = add_self_loops(pos_edge_index)
    #if neg_edge_index is None:
    #    neg_edge_index = negative_sampling(pos_edge_index, z.size(0))
    
    #neg_res, neg_reg = model.decoder(z, neg_edge_index, sigmoid=True)
    #neg_loss = -torch.log(1 - neg_res + EPS).mean()
    
    #loss_class = pos_loss+neg_loss
    #loss_reg = pos_reg+neg_reg

    #precision1 = torch.exp(-model.decoder.log_vars[0])
    #precision2 = torch.exp(-model.decoder.log_vars[1])

    #loss_class = precision1 *loss_class + model.decoder.log_vars[0]
    #loss_reg = precision2 *loss_reg + model.decoder.log_vars[1]

    #return F.binary_cross_entropy(logits, target_tensors[target_len])
    return criterion(logits, target_tensors[target_len], indexes)



In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, edge_features, use_edge_features = False):
        super(Decoder, self).__init__()
        self.features_size = len(edge_features)

        self.transfer0 = nn.Sequential(nn.Linear(input_size , hidden_size*2), nn.ELU(), nn.Dropout(0.05))


        #self.transfer1 = nn.Sequential(nn.Linear(hidden_size*2, hidden_size), nn.ELU(), nn.Dropout(drop_out),
        #                                                    nn.Linear(hidden_size, 1))

        self.transfer2 = nn.Sequential(nn.Linear(hidden_size*2, int(hidden_size)), nn.ELU(), nn.Dropout(0.03),
                            nn.Linear(int(hidden_size), 2))
        
        #self.log_vars = nn.Parameter(torch.zeros((2)))
        #self.loss_layer = torch.nn.SmoothL1Loss()
        #self.loss_layer = torch.nn.MSELoss()
        #self.loss_layer = torch.nn.L1Loss()
        self.use_edge_features = use_edge_features
        
    def forward(self, z, edge_index, sigmoid = True ):

        edge_index_np = edge_index.cpu().numpy()
        self.val_indices = x_edge_np[edge_index_np[0, :], edge_index_np[1, :]]
        self.val_indices = np.squeeze(np.asarray(self.val_indices))
        #vals = x_edge_vals_d[val_indices, :]

        #features = vals.to(dev)
        h1 = z[edge_index[0, :]]
        h2 = z[edge_index[1, :]]
        
        if self.use_edge_features :
            res = self.transfer0(torch.cat((h1, h2, features), dim=1))
            res = self.transfer1(res)
        else:
            h = torch.cat((h1, h2), dim=1)#, features), dim=1)
            transfered = self.transfer0(h)

            res = self.transfer2(transfered)
            res = torch.sigmoid(res) if sigmoid else res
            
            #res_reg = torch.sigmoid(self.transfer1(transfered))
            #reg_loss = self.loss_layer(res_reg, features)
        return res #reg_loss


In [ ]:
import pickle
import gnn_utils.graph_utils as gutil
importlib.reload(gutil)
sys.setrecursionlimit(100000)

train_verses = all_verses[:]
test_verses = all_verses[:]
editf1 = "eng-x-bible-mixed"
editf2 = 'fra-x-bible-louissegond'

small_editions = current_editions[:]
#if editf1 not in small_editions:
#    small_editions.append[editf1]
#if editf2 not in small_editions:
#    small_editions.append(editf2)

if 'jpn-x-bible-newworld' in small_editions:
    small_editions.remove('jpn-x-bible-newworld')
if 'grc-x-bible-unaccented' in small_editions:
    small_editions.remove('grc-x-bible-unaccented')


#train_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_eng_fra_full_community.pickle")
train_dataset, train_nodes_map = gutil.create_dataset(train_verses, verse_alignments_inter, small_editions)
torch.save(train_dataset, "/mounts/work/ayyoob/models/gnn/dataset_helfi_train_community.pickle")
features = train_dataset.features
train_nodes_map = train_dataset.nodes_map
#edge_index_intra_sent = train_dataset.edge_index_intra_sent
#test_edge_index_intra_sent = edge_index_intra_sent

# test_dataset, test_nodes_map = create_dataset(test_verses, verse_alignments_inter, small_editions)
test_dataset, test_nodes_map = train_dataset, train_nodes_map
test_verses = train_verses
print(train_dataset.x.shape, train_dataset.edge_index.shape, len(train_dataset.features))

# augment_features(test_dataset)

# x_edge, features_edge = create_edge_attribs(train_nodes_map, train_verses, small_editions, verse_alignments_inter, train_dataset.x.shape[0])
# with open("./dataset.pickle", 'wb') as of:
#     pickle.dump(train_dataset, of)

/mounts/Users/student/ayyoob/anaconda3/envs/multalign_graph/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
2021-08-31 19:39:35,337 - analytics - INFO - adding verses
2021-08-31 19:41:06,650 - analytics - INFO - augmenting node features
2021-08-31 19:41:46,842 - analytics - INFO - adding centrality features, nodes: 1219, edges: 23663
2021-08-31 19:41:46,917 - analytics - INFO - adding centrality features, nodes: 1682, edges: 27936
2021-08-31 19:41:46,976 - analytics - INFO - adding centrality features, nodes: 1980, edges: 42068
2021-08-31 19:41:46,982 - analytics - INFO - adding centrality features, nodes: 2090, edges: 53055
2021-08-31 19:41:46,983 - analytics - INFO - adding centrality featu

creating communities
creating communities


2021-08-31 19:43:11,480 - analytics - INFO - adding centrality features, nodes: 2122, edges: 45765
2021-08-31 19:43:13,949 - analytics - INFO - adding centrality features, nodes: 1135, edges: 24314


creating communities


2021-08-31 19:43:34,191 - analytics - INFO - adding centrality features, nodes: 1927, edges: 41992


...

creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities


2021-08-31 21:02:14,817 - analytics - INFO - adding centrality features, nodes: 1595, edges: 30055


creating communities
creating communities
creating communities
creating communities
creating communities
creating communities


2021-08-31 21:06:19,233 - analytics - INFO - adding centrality features, nodes: 965, edges: 21718


creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
creating communities
torch.Size([1452185, 9]) torch.Size([2, 57299834]) 9


In [ ]:
# edges_intra_sent, edges_seq_sent = get_inter_sentence_connections(train_dataset.nodes_map)
# edge_index_seq_sent = torch.tensor(edges_seq_sent, dtype=torch.long)
# train_dataset.edge_index_seq_sent = edge_index_seq_sent
# torch.cuda.set_device(int(free_gpu1))
# edge_index_intra_sent = torch.tensor(edges_intra_sent, dtype=torch.long)
# train_dataset.edge_index_intra_sent = edge_index_intra_sent
# test_edge_index_intra_sent = train_dataset.edge_index_intra_sent
# print(train_dataset.edge_index_intra_sent.shape)

In [ ]:
#import pickle
#import torch
## with open("./features_edge.pickle", 'wb') as of:
##     pickle.dump(features_edge, of)

## print('done first')
## with open("/mounts/work/ayyoob/models/gnn//x_edge.pickle", 'wb') as of:
##     pickle.dump(x_edge, of)

#with open("./features_edge.pickle", 'rb') as inf:
#    features_edge = pickle.load(inf)


## indices = [[],[]]
## values = []

## print('going to create sparse matrix representation')
## for i in range(len(ss)):
##     print(i)
##     for j in range(len(ss)):
##         if ss[i][j] != None and ss[i][j] != []:
##             indices[0].append(i)
##             indices[1].append(j)
##             values.append(ss[i][j])


## with open("./edge_attribs_sparse_indices.pickle", 'wb') as of:
##     pickle.dump(indices, of)

## with open("./edge_attribs_sparse_values.pickle", 'wb') as of:
##     pickle.dump(values, of)

## print('loading indices')
## with open("./edge_attribs_sparse_indices.pickle", 'rb') as inf:
##     indices = pickle.load(inf)

## print('loading values')
## with open("./edge_attribs_sparse_values.pickle", 'rb') as inf:
##     values = pickle.load(inf)

## print('creating sparse tensor')
## s = torch.sparse_coo_tensor(indices, values, (67800, 67800, len(ff)), dtype=torch.float16)
## print('saving sparse matrix')
## torch.save(s, "/mounts/work/ayyoob/models/gnn/edge_attribs_tensor16.pickle")

#print('loading sparse matrix')
#x_edge = torch.load("/mounts/work/ayyoob/models/gnn/edge_attribs_tensor.pickle")


#train_dataset.features_edge = features_edge



In [ ]:
#from scipy.sparse import csr_matrix
#import numpy as np

#x_edge = x_edge.coalesce()
#torch.cuda.set_device(1)
#x_edge_vals = x_edge.values()
#indices_np = x_edge.indices().numpy()
#print(indices_np.shape)
#x_edge_np = csr_matrix((np.arange(indices_np.shape[1]), (indices_np[0, :], indices_np[1,:])), shape=(67800, 67800))
#print("done")

In [ ]:
##x_edge_vals = x_edge_vals.cpu()
##maxes = torch.max(x_edge_vals,0)
##mins = torch.min(x_edge_vals,0)
##x_edge_vals_d = torch.div(x_edge_vals, maxes.values)

#targets = torch.zeros(indices_np.shape[1], dtype=torch.int64)
#pos_indices = x_edge_np[train_dataset.edge_index.cpu().numpy()[0,:], train_dataset.edge_index.cpu().numpy()[1,:]]
#pos_indices = np.squeeze(np.asarray(pos_indices))
#targets[pos_indices] = 1

In [ ]:
#run on delta, extract w2v features
sys.path.insert(0, '../')
import pickle
from gensim.models import Word2Vec
from app import document_retrieval
from my_utils import utils
config_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc-ui-demo/config_pbc.ini"
utils.setup(config_file)
import torch
import my_utils.alignment_features as feat_utils
importlib.reload(document_retrieval)

doc_retriever = document_retrieval.DocumentRetriever()

model_w2v = Word2Vec.load("/mounts/work/ayyoob/models/w2v/word2vec_helfi_langs_15e.model")
#train_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_hib_fin_full.pickle")
nodes_map = train_dataset.nodes_map

x = [[] for i in range(train_dataset.x.shape[0])]
for edition_f in nodes_map:
    utils.LOG.info(f"processing edition {edition_f}")
    for verse in nodes_map[edition_f]:         #toknom nodecount
        line = doc_retriever.retrieve_document(f'{verse}@{edition_f}')
        line = line.strip().split()

        for tok in nodes_map[edition_f][verse]:
            w_emb = model_w2v.wv[f'{edition_f[:3]}:{line[tok]}']
            x[nodes_map[edition_f][verse][tok]].extend(w_emb)

x = torch.tensor(x, dtype=torch.float)
train_dataset.x = torch.cat((train_dataset.x, x), dim=1)
train_dataset.features.append(feat_utils.ForwardFeature(100, 100, 'W2v'))

print(x.shape, train_dataset.x.shape, len(train_dataset.features))

torch.save(train_dataset, "/mounts/work/ayyoob/models/gnn/dataset_helfi_train_community.pickle")

2021-08-31 21:14:20,872 - analytics - INFO - processing edition eng-x-bible-mixed
2021-08-31 21:14:20,872 - analytics - INFO - processing edition eng-x-bible-mixed
2021-08-31 21:14:23,572 - analytics - INFO - processing edition deu-x-bible-newworld
2021-08-31 21:14:23,572 - analytics - INFO - processing edition deu-x-bible-newworld
2021-08-31 21:14:26,350 - analytics - INFO - processing edition ces-x-bible-newworld
2021-08-31 21:14:26,350 - analytics - INFO - processing edition ces-x-bible-newworld
2021-08-31 21:14:28,971 - analytics - INFO - processing edition fra-x-bible-louissegond
2021-08-31 21:14:28,971 - analytics - INFO - processing edition fra-x-bible-louissegond
2021-08-31 21:14:31,638 - analytics - INFO - processing edition ita-x-bible-2009
2021-08-31 21:14:31,638 - analytics - INFO - processing edition ita-x-bible-2009
2021-08-31 21:14:34,176 - analytics - INFO - processing edition spa-x-bible-newworld
2021-08-31 21:14:34,176 - analytics - INFO - processing edition spa-x-bib

torch.Size([1452185, 100]) torch.Size([1452185, 109]) 10


In [ ]:
len(train_dataset.nodes_map.keys())

81

In [ ]:
from gnn_utils.eval_utils import *
train_dataset.train_mask = train_dataset.val_mask = train_dataset.test_mask = train_dataset.y = None
test_dataset.train_mask = test_dataset.val_mask = test_dataset.test_mask = test_dataset.y = None

div1 = 1
div2 = 2
torch.cuda.set_device(div1)
print("device: ", dev)
# x_test, test_dataset.edge_index, test_dataset.neg_edge_index = test_dataset.x.to(dev), test_dataset.edge_index.to(dev), test_dataset.neg_edge_index.to(dev)
x = train_dataset.x.to(dev)
# edge_index_intra_sent, edge_index_seq_sent = train_dataset.edge_index_intra_sent.to(dev), train_dataset.edge_index_seq_sent.to(dev)
edge_index_intra_sent, edge_index_seq_sent = torch.tensor([[],[]], dtype=torch.long).to(dev), train_dataset.edge_index_seq_sent.to(dev)
test_edge_index_intra_sent = edge_index_intra_sent

train_pos_edge_index = train_dataset.edge_index.to(dev)
x_test, test_dataset.edge_index = x, train_pos_edge_index
target_tensors = {}
targets = targets.to(dev)


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
#from tqdm import tqdm
#gc.collect()
#with torch.no_grad():
#    torch.cuda.empty_cache()

#torch.cuda.set_device(div1)
#run += 1
#start_prune = 120
#features_edge = [] #TODO remove me
#features_edge = train_dataset.features_edge
##features = train_dataset.features[:2]

#drop_out = 0
#pos_noise = 0.0
#neg_noise = 0.0
#n_head = 1


#batch_size = 10000
#channels = 48

#in_dim = sum(t.out_dim for t in features)
#decoder_in_dim = channels * 2 * n_head + len(features_edge) #+ sum(t.out_dim for t in features_edge)
#print('edge features size: ', sum(t.out_dim for t in features_edge))
#decoder = Decoder(n_head * channels *2, int(decoder_in_dim/2), features_edge)
#model = pyg_nn.GAE(Encoder(in_dim, channels, features, n_head, edge_feature_dim=len(features_edge)), decoder=decoder).to(dev)

#print("sending input to gpu")
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
#criterion = TruncatedLoss(q=0.1,k=0.49,trainset_size=x_edge_vals.shape[0]).to(dev)
#writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S-") + f"samett-{channels}chs-feat{train_dataset.num_node_features}-")

#torch.set_printoptions(edgeitems=5)
#print("model params - decoder params - conv1", sum(p.numel() for p in model.parameters()), sum(p.numel() for p in decoder.parameters()))
#best_f1 = 0

for epoch in range(1, 200):

    train_neg_edge_index = get_negative_edges(train_verses, small_editions, train_dataset.nodes_map,  verse_alignments_inter).to(dev)
    #edge_index_seq_sent_neg = get_negative_edges_seq(train_dataset.nodes_map).to(dev)

    train(epoch)
    print("epoch", epoch)
    if epoch % 1 == 0:
        f1 = eval_utils.alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses, test_nodes_map,
                        dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer)

        if f1 > best_f1:
            best_f1 = f1
            checkpoint(f1, epoch, model, run)
        if not os.path.isdir('/mounts/work/ayyoob/models/gnn/checkpoint'):
            os.mkdir('/mounts/work/ayyoob/models/gnn/checkpoint')

        state = {
            'current_net': model,
        }
        torch.save(state, '/mounts/work/ayyoob/models/gnn/checkpoint/current_net')
        state["current_net"] = None

    gc.collect()
    with torch.no_grad():   
        torch.cuda.empty_cache()

# if epoch == 4:
#     optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
torch.cuda.set_device(div1)

from gnn_utils import eval_utils
importlib.reload(eval_utils)
eval_utils.alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses, test_nodes_map,
                        dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer)

In [ ]:
global model, decoder
1/0
decoder = None
model = None

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
#features = train_dataset.features[:]
#features_edge = train_dataset.features_edge[:]
from pprint import pprint
print('indim',in_dim)
#features[-1].out_dim = 50
for i in features_edge:
    #if i.type==3:
    #    i.out_dim=4
    print(vars(i))

#sum(p.out_dim for p in features)
#features[-1].out_dim = 10


In [ ]:
# count number of deleted edges by each community detection method
# from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

# tmp_verses = [all_verses[2]]
# tmp_editions = small_editions[:10]
# tmp_dataset, tmp_nodes_map = create_dataset(tmp_verses, verse_alignments_inter, tmp_editions)

# tmp_g = pyg_utils.convert.to_networkx(tmp_dataset, to_undirected=True)
# def count_deleted_edges(tmp_dataset, c):
#     deleted_edges = 0
#     for i in range(0, len(tmp_dataset.edge_index[0]), 2):
#         for comp in c:
#             if tmp_dataset.edge_index[0][i].item() in comp and tmp_dataset.edge_index[1][i].item() not in comp:
#                 deleted_edges += 1
    
#     return deleted_edges

# print("eng token count: ", tmp_nodes_map['eng-x-bible-mixed'][tmp_verses[0]])
# print("original connected components",nx.number_connected_components(tmp_g))

# c = list(greedy_modularity_communities(tmp_g))
# print("new connected_components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# c = list(asyn_lpa_communities(tmp_g))
# print("asyn_lpa_communities number of components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# c = list(label_propagation_communities(tmp_g))
# print("label_propagation_communities number of components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# cents = nx.edge_betweenness_centrality(tmp_g)
# vals = sorted(list(cents.values()))
# print(vals[0], vals[10], vals[100], vals[1000], vals[2000], vals[3000], vals[10000])
# print(vals[-1], vals[-10], vals[-100], vals[-1000], vals[-2000], vals[-3000], vals[-10000])



In [ ]:
# measure different community detection algorithms
# from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

# def remove_bad_community_edges(nodes_map, verses, edition_files, alignments):
#     edges_tmp = [[],[]]
#     res_edges = [[],[]]
#     for verse in verses:
#         utils.LOG.info(f"extracting edge features for {verse}")
#         for i,editf1 in enumerate(edition_files):
#             for j,editf2 in enumerate(edition_files[i+1:]):
#                 aligns = autils.get_aligns(editf1, editf2, alignments[verse])
#                 if aligns != None:
#                     for align in aligns:
#                         n1, node_count = node_nom(verse, editf1, align[0], 0, nodes_map, None, None)
#                         n2, node_count = node_nom(verse, editf2, align[1], 0, nodes_map, None, None)
#                         edges_tmp[0].extend([n1, n2])
#                         edges_tmp[1].extend([n2, n1])

#         gnx = convert_to_netx(edges_tmp)
#         print('detecting communities')
#         coms = greedy_modularity_communities(gnx)

#         print('finding good edges')
#         for i in range(0, len(edges_tmp[0]), 2):
#             for c in coms:
#                 if edges_tmp[0][i] in c and edges_tmp[0][i+1] in c:
#                     res_edges[0].extend([edges_tmp[0][i], edges_tmp[0][i+1]])
#                     res_edges[1].extend([edges_tmp[0][i+1], edges_tmp[0][i]])
#         edges_tmp = [[],[]]
#     print('to keep edges:', len(res_edges[0]))
#     return torch.tensor(res_edges, dtype=torch.long)

# # old_edge_index = train_dataset.edge_index
# # new_edge_index = remove_bad_community_edges(train_dataset.nodes_map, train_verses, small_editions, verse_alignments_inter)
# # train_dataset.edge_index = new_edge_index

# # with open("./dataset_greedy_modularity_communities.pickle", 'rb') as inf:
# #     train_dataset = pickle.load(inf)

# test_dataset = train_dataset

# print('orig edge count', old_edge_index.shape)
# print('new edge count', train_dataset.edge_index.shape)
# print("done")

In [ ]:
nodes_map = train_dataset.nodes_map
bad_edition_files = []
for edit in nodes_map:
    bad_count = 0
    for verse in nodes_map[edit]:
        if len(nodes_map[edit][verse].keys()) < 6:
            bad_count += 1
        if bad_count > 1:
            bad_edition_files.append(edit)
            break
print(bad_edition_files)

In [ ]:
all_japanese_nodes = set()
nodes_map = train_dataset.nodes_map

for verse in nodes_map['jpn-x-bible-newworld']:
    for item in nodes_map['jpn-x-bible-newworld'][verse].items():
        all_japanese_nodes.add(item[1])

print(" all japansese nodes: ", len(all_japanese_nodes))
edge_index = train_dataset.edge_index.to('cpu')
remaining_edges_index = []
for i in tqdm(range(0, edge_index.shape[1], 2)):
    if edge_index[0, i].item() not in all_japanese_nodes and edge_index[0, i+1].item() not in all_japanese_nodes:
        remaining_edges_index.extend([i, i+1])

print('original total edges count', edge_index.shape)
print('remaining edge count', len(remaining_edges_index))
train_dataset.edge_index = edge_index[:, remaining_edges_index]
train_dataset.edge_index.shape


In [ ]:
from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

def get_community_edges(c, verse):
    res = []
    for n1 in all_nodes_map[editf1][verse].items():
        for n2 in all_nodes_map[editf2][verse].items():
            for com in c:
                if n1[1] in com and n2[1] in com:
                    res.append((n1[0], n2[0]))
    
    return res

def intersect(e1, e2):
    res = set()
    for item in e1:
        if item in e2:
            res.add(item)
    
    return res


measures = {}
measures['intersection']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['gdfa']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c1_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c1_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c2_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c2_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c3_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c3_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
numbers = [100, 300, 600, 1000, 1300, 1600, 2000, 3000, 4000, 5000]
for i in numbers:
    measures[i] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
    measures[i+1] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
coms = {}
for verse in test_verses:
    inter_edges = autils.get_aligns(editf1, editf2, verse_alignments_inter[verse])

    all_dataset, all_nodes_map = create_dataset([verse], verse_alignments_inter, small_editions)
    print("converting")
    g = pyg_utils.convert.to_networkx(all_dataset, to_undirected=True)
    print("detecting community 1")
    c1 = list(greedy_modularity_communities(g))
    print("detecting community 2")
    c2 = list(asyn_lpa_communities(g))
    print("detecting community 3")
    c3 = list(label_propagation_communities(g))

    print("detecting community 4")
    edge_betweenness = [y[0] for y in sorted(nx.edge_betweenness_centrality(g).items(), key=lambda x: x[1], reverse=True)]
    
    print('orig communities', nx.number_connected_components(g))
    prev_i = 0
    for i in numbers:
        for j in range(prev_i, i):
            g.remove_edge(*edge_betweenness[j])
        prev_i = i
        com = list(nx.connected_components(g))
        edges = get_community_edges(com, verse)
        autils.calc_and_update_alignment_score(edges, pros[verse], surs[verse], measures[i])
        autils.calc_and_update_alignment_score(intersect(edges, inter_edges), pros[verse], surs[verse], measures[i+1]) 
        print(f'communities {i}', nx.number_connected_components(g))

    c1_edges = get_community_edges(c1, verse)
    c2_edges = get_community_edges(c2, verse)
    c3_edges = get_community_edges(c3, verse)
    print('c1 communities', len(c1))
    print('c2 communities', len(c2))
    print('c3 communities', len(c3))



    autils.calc_and_update_alignment_score(inter_edges, pros[verse], surs[verse], measures['intersection'])
    autils.calc_and_update_alignment_score(autils.get_aligns(editf1, editf2, verse_alignments_gdfa[verse]), pros[verse], surs[verse], measures['gdfa'])

    autils.calc_and_update_alignment_score(c1_edges, pros[verse], surs[verse], measures['c1_all'])
    autils.calc_and_update_alignment_score(c2_edges, pros[verse], surs[verse], measures['c2_all'])
    autils.calc_and_update_alignment_score(c3_edges, pros[verse], surs[verse], measures['c3_all'])

    autils.calc_and_update_alignment_score(intersect(c1_edges, inter_edges), pros[verse], surs[verse], measures['c1_inter'])
    autils.calc_and_update_alignment_score(intersect(c2_edges, inter_edges), pros[verse], surs[verse], measures['c2_inter'])
    autils.calc_and_update_alignment_score(intersect(c3_edges, inter_edges), pros[verse], surs[verse], measures['c3_inter'])

    for item in measures:
        print(item, measures[item])
    
    
